# Calculating Max Flow with 1 Simple Example

In this notebook, we'll work through an implementation of Maximum Flow (**Max Flow**) on a simple directed graph. I will be following the Ford-Fulkerson approach towards this implementation. 

Maximum Flow refers to the carrying capacity between two nodes in a directed graph. These two nodes are termed the *source* and *target* here. Flow originates at the source, and terminates at the target. 

References: 
* Ford, L. R.; Fulkerson, D. R. (1956). *"Maximal flow through a network"*. Canadian Journal of Mathematics. 8: 399–404
*  Edmonds, Jack; Karp, Richard M. (1972). *"Theoretical improvements in algorithmic efficiency for network flow problems"*. Journal of the ACM. 19 (2): 248–264

In [1]:
import numpy as np
import pandas as pd
from typing import Tuple, List

## Ford-Fulkerson - Edmonds-Karp Algorithm

In the associated blog article, I illustrate the concept of max flow. Here it is sufficient to state that the algorithm we will implement is as follows:
1. Start with a directed graph, with each edge having a positive *capacity* $C_{edge}$ value associated with it. I assume that there is only 1 edge going from *source* to *target*
2. Prepare a *residual graph*: this involves adding new edges to the graph with reversed direction, and an initial capacity of 0 
3. Use BFS to identify a path through the residual graph from *source* to *target*, where the capacity for all edges involved is > 0
4. Determine the minimum capacity $C_{min}$ along the discovered path
5. Update the residual graph in the following way:
    * For all edges (*source* to *target*) in path update the capacity as follows: $C_{edge} \leftarrow C_{edge} - C_{min}$
    * For all edges (*target* to *source*), update the capacity as follows: $C_{edge} \leftarrow C_{edge} + C_{min}$
6. Go back to step 3 above, and repeat for as along as there is a viable path from *source* to *target*

## Implementation

In [2]:
class MaxFlowGraph(object):

    def __init__(self, edges: List[Tuple]) -> None:
        self.edges = edges
        nodes = set()
        for edge in edges:
            nodes.update(list(edge)[:-1])
        self.nodes = list(nodes)
        self.nodes.sort()
        self.number_nodes = len(self.nodes)
        self.reset_residual_graph()
        
    def reset_residual_graph(self) -> None:
        self.residual_graph = self.edges.copy()
        count = len(self.residual_graph)
        for i in range(count):
            edge = self.residual_graph[i]
            self.residual_graph.append((edge[1],edge[0],0,))

    def get_adjacency_matrix(self) -> np.array:
        # work out adjacency matrix
        O = np.zeros((self.number_nodes, self.number_nodes))
        for edge in self.residual_graph:
            src, dst, capacity = edge
            if capacity > 0:
                O[self.nodes.index(dst),self.nodes.index(src)] = 1
        return O

    def compute_residual_capacity_on_edge(self, src: str, dst: str, reduction_amount: float) -> None:
        # here I assume there is only 1 edge from src to dst
        for i, edge in enumerate(self.residual_graph):
            edge_src, edge_dst, edge_capacity = edge
            if (src == edge_src) and (dst == edge_dst) and (edge_capacity > 0):
                self.residual_graph[i] = (edge_src, edge_dst, edge_capacity - reduction_amount,)
            elif (src == edge_dst) and (dst == edge_src):
                self.residual_graph[i] = (edge_src, edge_dst, edge_capacity + reduction_amount,)

In [3]:
class BFS(object):

    def __init__(self, graph: MaxFlowGraph) -> None:
        self.graph = graph

    def _prepare_for_traversal(self, source: str) -> None:
        self.queue = [source]
        self.visited = dict(zip(self.graph.nodes, len(self.graph.nodes) * [False]))
        self.visited[source] = True

    def _process_unvisited_neighbours(self, node: str) -> None:
        node_index = self.graph.nodes.index(node)
        neighbours_indices = self.graph.get_adjacency_matrix()[:,node_index].nonzero()[0].tolist()
        neighbours = np.array(self.graph.nodes)[neighbours_indices].tolist()
        for neighbour in neighbours:
            if not self.visited[neighbour]:
                self.queue.append(neighbour)
                self.visited[neighbour] = True
    
    def traverse(self, source: str) -> None:
        # prepare for the traversal
        self._prepare_for_traversal(source)
        # process queue
        while self.queue:
            # pop a node from the queue
            node = self.queue.pop(0)
            # go to each unvisited neighbour of node
            self._process_unvisited_neighbours(node)

In [4]:
class Path(object):

    def __init__(self, label: str):
        self.label = label
        self.next_level = []

class PathFinder(BFS):
    
    def _add_to_paths(self, paths: Path, node: str, neighbour: str) -> None:
        if paths.label == node:
            paths.next_level.append(Path(neighbour))
        elif (paths.label != node) and paths.next_level:
            for path in paths.next_level:
                self._add_to_paths(path, node, neighbour)    
    
    def _process_unvisited_neighbours(self, node: str) -> None:
        node_index = self.graph.nodes.index(node)
        neighbours_indices = self.graph.get_adjacency_matrix()[:,node_index].nonzero()[0].tolist()
        neighbours = np.array(self.graph.nodes)[neighbours_indices].tolist()
        for neighbour in neighbours:
            if not self.visited[neighbour]:
                self.queue.append(neighbour)
                self.visited[neighbour] = True
                self._add_to_paths(self.paths, node, neighbour)

    def _determine_shortest_path(self, paths: Path, target: str, path_string: str) -> None:
        if paths.label == target:
            self.shortest_path = path_string + target
        elif paths.next_level:
            for path in paths.next_level:
                self._determine_shortest_path(path, target, path_string + paths.label)

    def find_shortest_path(self, source: str, target: str) -> bool:
        # initialize path string
        self.shortest_path = ""
        # define a Path instance to store all possible paths from source
        self.paths = Path(source)
        # traverse graph
        self.traverse(source)
        # identify shortest path that terminates at target
        self._determine_shortest_path(self.paths, target, self.shortest_path)
        # return status
        if self.shortest_path:
            return True
        return False

In [5]:
# iterate over all possible paths, compute max flow
def maxflow(pf: PathFinder, src: str, dst: str) -> float:
    
    maxflow = 0
    while pf.find_shortest_path(src, dst):
        # prepare path
        path = pf.shortest_path
        path = list(path)
    
        # get minimum capacity along path
        dfEdges = pd.DataFrame(pf.graph.residual_graph, columns=['src', 'dst', 'capacity'])
        min_capacity = float('inf')
        for i in range(len(path)-1):
            row = dfEdges[(dfEdges.src == path[i]) & (dfEdges.dst == path[i+1])]
            if row.capacity.values[0] < min_capacity:
                min_capacity = row.capacity.values[0]

        # max flow
        maxflow += min_capacity
    
        # compute residual graph
        for i in range(len(path)-1):
            pf.graph.compute_residual_capacity_on_edge(path[i], path[i+1], min_capacity)
    
    return maxflow

## Test Implementation

The example I will work through here is based off of the one outlined here: https://theory.stanford.edu/~tim/w16/l/l1.pdf

In [6]:
edges = [("A", "B", 3),("B", "D", 2),("A", "C", 2),("B", "C", 5),("C", "D", 3)]

In [7]:
# create a graph instance
graph = MaxFlowGraph(edges)

In [8]:
# view initial residual graph
graph.residual_graph

[('A', 'B', 3),
 ('B', 'D', 2),
 ('A', 'C', 2),
 ('B', 'C', 5),
 ('C', 'D', 3),
 ('B', 'A', 0),
 ('D', 'B', 0),
 ('C', 'A', 0),
 ('C', 'B', 0),
 ('D', 'C', 0)]

In [9]:
# view detected nodes
graph.nodes

['A', 'B', 'C', 'D']

In [10]:
# view initial adjacency matrix
graph.get_adjacency_matrix()

array([[0., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [0., 1., 1., 0.]])

In [11]:
# create a PathFinder instance
pf = PathFinder(graph)

In [12]:
# compute maxflow
maxflow(pf, 'A', 'D')

5

In [13]:
# view final residual graph
graph.residual_graph

[('A', 'B', 0),
 ('B', 'D', 0),
 ('A', 'C', 0),
 ('B', 'C', 4),
 ('C', 'D', 0),
 ('B', 'A', 3),
 ('D', 'B', 2),
 ('C', 'A', 2),
 ('C', 'B', 1),
 ('D', 'C', 3)]

In [14]:
# view final adjacency matrix
graph.get_adjacency_matrix()

array([[0., 1., 1., 0.],
       [0., 0., 1., 1.],
       [0., 1., 0., 1.],
       [0., 0., 0., 0.]])

## Conclusions

We can verify above that our implementation works in obtaining the correct max flow value for the graph tested. We can also check to see the changes that occure to the residual graph, and adjacency matrix, after running the algorithm. These changes are an expected part of this implementation.

Note to rerun maxflow, it is required to reset the residual graph. This can be done via the *reset_residual_graph* function.